# EE2073 Lab 2
### Author: Nguyen Hoang Khanh (Kelvin)

#### Question 4.2

Communication channel setup

In [1]:
import serial
import serial.tools.list_ports

VID = 61525
PID = 38912

ports = serial.tools.list_ports.comports()

device = None
for p in ports:
    if p.vid == VID and p.pid == PID:
        try:
            device = serial.Serial(p.device)
        except serial.SerialException:
            print('Reconnect the controller unit.')

if device is None:
    raise Exception('No suitable device detected.')

Set the VDCP supply voltage

In [ ]:
vdc = 12 # The expected DC supply voltage
cmd_setvdc = 'dz' + str(int(vdc*100)).zfill(4) + '\r'

Send command string to the MCU

In [ ]:
device.write(bytes(cmd_setvdc, 'utf-8'))

Observation:\
Connected to VDCP: measured V = 11.58 (V)\
Connected to VDCN: measured V = -11.31 (V)

Close the connection

In [22]:
device.close()

Open the connection (after closing)

In [ ]:
device.open()

#### Question 4.3

Generate sinusoidal signal waveform

In [2]:
ns = 64             # samples per cycle
freq = 2000         # frequency (Hz)
amp = 0.5           # amplitude (Vpk)
offset = 0          # offset (DC V)
cmd_gensin = 's100' # W1 sinusoidal
cmd_gensin += str(ns).zfill(3) + str(freq).zfill(7) + str(int(amp*100)).zfill(4) + str(int(offset*100)).zfill(4) + '\r'

device.write(bytes(cmd_gensin, 'utf-8'))

23

Observation:\
Measured Vrms = 0.318 (V)\
Theoretical Vrms = 0.707 * Vpk = 0.707 * 0.5 = 0.3535 (V)\
Discrepancy is due to residual offset from the DC bias, which will be addressed in latter parts of the course.

Generate triangular waveform

In [ ]:
# Reuse the same amp, freq and offset
cmd_gentri = 's110' # W1 triangular
cmd_gentri += str(ns).zfill(3) + str(freq).zfill(7) + str(int(amp*100)).zfill(4) + str(int(offset*100)).zfill(4) + '\r'

device.write(bytes(cmd_gentri, 'utf-8'))

Observation:\
Measured Vrms = 0.258 (V)\
Theoretical Vrms = Vpk / sqrt(3) = 0.2887 (V)\
Reference: https://testbook.com/question-answer/for-rms-100-value-triangular-wave-the-peak-voltag--5ef958d9749d540d095ceed3 \
Discrepancy is also due to residual offset.


#### Question 4.4

Reading timed voltage signal

In [15]:
fs = 200000     # sampling frequency (Hz)

c1 = 0          # CH1 DC coupling mode; 0 for signal within around +- 1V
gain1 = 138     # CH1 gain; set the range of voltage that can be read
dco1 = 130      # CH1 offset; set the zero level

c2 = 0          # CH2 DC coupling mode
gain2 = 138     # CH2 gain
dco2 = 130      # CH2 offset

cmd_readosc = 'm1' + str(fs).zfill(6) + str(c1) + str(gain1).zfill(3) + str(dco1).zfill(3) + \
                                        str(c2) + str(gain2).zfill(3) + str(dco2).zfill(3) + '\r'

# Store sampled data into byte array
bytedata = bytearray(4000)      # create a byte array of size 4000
device.reset_input_buffer()     # clear data in input buffer
device.write(bytes(cmd_readosc, 'utf-8'))
device.readline()               # skip line containing the cmd string
device.readinto(bytedata)

4000

Convert byte array into 2-D NumPy array

In [16]:
import numpy as np

In [17]:
data = np.frombuffer(bytedata, dtype='uint16').reshape((2, 1000))

Showing the NumPy array `data`

In [18]:
data

array([[2539, 2538, 2531, ..., 2510, 2546, 2531],
       [2531, 2510, 2487, ..., 2523, 2550, 2516]], dtype=uint16)

In [19]:
data.shape

(2, 1000)

#### Question 4.5
Visualize timed voltage signals

Create time array

In [11]:
t = np.arange(1000)/fs

In [12]:
t

array([0.000e+00, 5.000e-06, 1.000e-05, 1.500e-05, 2.000e-05, 2.500e-05,
       3.000e-05, 3.500e-05, 4.000e-05, 4.500e-05, 5.000e-05, 5.500e-05,
       6.000e-05, 6.500e-05, 7.000e-05, 7.500e-05, 8.000e-05, 8.500e-05,
       9.000e-05, 9.500e-05, 1.000e-04, 1.050e-04, 1.100e-04, 1.150e-04,
       1.200e-04, 1.250e-04, 1.300e-04, 1.350e-04, 1.400e-04, 1.450e-04,
       1.500e-04, 1.550e-04, 1.600e-04, 1.650e-04, 1.700e-04, 1.750e-04,
       1.800e-04, 1.850e-04, 1.900e-04, 1.950e-04, 2.000e-04, 2.050e-04,
       2.100e-04, 2.150e-04, 2.200e-04, 2.250e-04, 2.300e-04, 2.350e-04,
       2.400e-04, 2.450e-04, 2.500e-04, 2.550e-04, 2.600e-04, 2.650e-04,
       2.700e-04, 2.750e-04, 2.800e-04, 2.850e-04, 2.900e-04, 2.950e-04,
       3.000e-04, 3.050e-04, 3.100e-04, 3.150e-04, 3.200e-04, 3.250e-04,
       3.300e-04, 3.350e-04, 3.400e-04, 3.450e-04, 3.500e-04, 3.550e-04,
       3.600e-04, 3.650e-04, 3.700e-04, 3.750e-04, 3.800e-04, 3.850e-04,
       3.900e-04, 3.950e-04, 4.000e-04, 4.050e-04, 

In [13]:
t.shape

(1000,)

Visualize the data (from CH1)

In [20]:
import plotly

fig = plotly.graph_objs.Figure()
fig.add_trace(plotly.graph_objs.Scatter(x=t*1e3, y=data[0, :])) # y = get data from CH1
fig.update_layout(xaxis_title='Time (ms)', yaxis_title='Voltage (a.u.)')

Visualize the data from CH2

In [21]:
fig_ch2 = plotly.graph_objs.Figure()
fig_ch2.add_trace(plotly.graph_objs.Scatter(x=t*1e3, y=data[1, :])) # y = get data from CH2
fig_ch2.update_layout(xaxis_title='Time (ms)', yaxis_title='Voltage (a.u.)')

Comment: waveforms from both CH1 and CH2 are identical.\
However, the waveforms are not exactly sinusoidal. Perhaps the quality is not perfect because the sampling rate is not high enough.

### Open-ended questions

- Sampling rate should be at least twice of the frequency (`Nyquist rate`, reference: https://www.techtarget.com/whatis/definition/Nyquist-Theorem#:~:text=The%20Nyquist%20rate%20is%202x,important%20part%20of%20information%20theory). Therefore, with maximum sampling rate of 210 kHz, the maximum signal frequency should be `210 / 2 = 105 (kHz)` for precise sampling.
- If the ADC tries to pick up a signal higher than that, the sampling is no more accurate. There will be a phenomenon called `aliasing` (reference: https://www.kistler.com/INT/en/sampling-rate-and-aliasing-effect/C00000151). As a result, the sampled signal will experience undersampling and appear as lower frequency and incorrect amplitude.
- Time span = (# data points) / (sampling rate) = 10^3 / (2 * 10^5) = 5 * 10^-3 (s) = `5 (ms)` (reference: https://www.ni.com/docs/en-US/bundle/labwindows-cvi/page/advancedanalysisconcepts/lvac_nsamples.html?srsltid=AfmBOop0azzzgSdHwnf_UmBjyWIjm49ItaBYzT5_zuksFvH3D5dBsSmk)